### Análisis de Datos del Mercado de Criptomonedas con Binance API

- En la era digital actual, el mercado de criptomonedas ha emergido como un campo fascinante y dinámico, ofreciendo oportunidades únicas para el análisis de datos y la comprensión de las tendencias del mercado financiero. Nuestro proyecto se centra en explorar este sector vibrante utilizando una de las herramientas más poderosas en el mundo de las criptomonedas: la API de Binance.

- Binance, siendo una de las plataformas de intercambio de criptomonedas más grandes y reconocidas a nivel mundial, proporciona una API rica y robusta, permitiendo a los usuarios acceder a una amplia gama de datos del mercado en tiempo real. Esto incluye, pero no se limita a, datos históricos de precios, detalles de operaciones individuales, volúmenes de trading, y mucho más.

- El objetivo principal de este proyecto es aprovechar estos datos para obtener insights valiosos sobre el comportamiento del mercado de Bitcoin (BTC), la criptomoneda líder en el mercado. Nos centraremos en extraer datos históricos de precios y operaciones de BTC comparados con el USD, analizando las tendencias, volatilidades y posibles patrones de trading.

- Al final de este proyecto, esperamos tener un análisis detallado y multifacético del mercado de Bitcoin, brindando un panorama claro no solo para los entusiastas de las criptomonedas, sino también para aquellos interesados en las finanzas y la tecnología. Este desafío es una excelente oportunidad para sumergirnos en el mundo del análisis de datos financieros y abrir la puerta a futuras investigaciones y proyectos en este apasionante campo.

In [1]:
# paquetes necesarios
import requests
import pandas as pd
import hmac
import hashlib
import time

In [2]:
api_key = '4BfPXEo2L55fb4WFe6EaNVVCl3JBuCyrEn75lM3y6R1Ut7HiLqTKBfBe5wquc85d'
api_secret = '9MZFj7DhnZTOViXtjMq7oupWg7aaN4bx7Xc7qb4ezPL0MOQt0IXKHLylYngFFPA0'
api_limit = 1000

# Función para firmar los parámetros
def sign_request(api_secret, params):
    query_string = '&'.join(["{}={}".format(d, params[d]) for d in params])
    signature = hmac.new(api_secret.encode(), query_string.encode(), hashlib.sha256).hexdigest()
    return signature

# Obtener operaciones históricas
def get_historical_trades(api_key, api_secret):
    url = "https://api.binance.com/api/v3/historicalTrades"
    params = {
        'symbol': 'BTCUSDT',
        'limit': api_limit
    }
    headers = {'X-MBX-APIKEY': api_key}
    response = requests.get(url, headers=headers, params=params)

    if response.status_code != 200:
        print(f"Error en la solicitud de operaciones históricas: Código de estado {response.status_code}")
        print(response.text)
        return None
    else:
        return pd.DataFrame(response.json())

# Obtener precios históricos
def get_historical_prices(api_key, api_secret):
    url = "https://api.binance.com/api/v3/klines"
    params = {
        'symbol': 'BTCUSDT',
        'interval': '1h',
        'limit': api_limit
    }
    headers = {'X-MBX-APIKEY': api_key}
    response = requests.get(url, headers=headers, params=params)

    if response.status_code != 200:
        print(f"Error en la solicitud de precios históricos: Código de estado {response.status_code}")
        print(response.text)
        return None
    else:
        data = response.json()
        columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 'Number of Trades', 'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore']
        df = pd.DataFrame(data, columns=columns)
        df['Open Time'] = pd.to_datetime(df['Open Time'], unit='ms')
        df['Close Time'] = pd.to_datetime(df['Close Time'], unit='ms')
        return df

df_trades = get_historical_trades(api_key, api_secret)
df_prices = get_historical_prices(api_key, api_secret)

## Unificar los 2 dataFrames en uno

In [3]:
df_trades['Time Interval'] = pd.to_datetime(df_trades['time'], unit='ms').dt.floor('H')

# conbinacion de dataframes
# Usamos un merge left para mantener todas las filas de df_trades y agregar la información correspondiente de df_prices
combined_df = pd.merge(df_trades, df_prices, left_on='Time Interval', right_on='Open Time', how='left')
combined_df.head()

,id,price,qty,quoteQty,time,isBuyerMaker,isBestMatch,Time Interval,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Ignore
0,3393178220,41765.99000000,0.00047000,19.63001530,1706473247780,False,True,2024-01-28 20:00:00,2024-01-28 20:00:00,41785.91000000,41854.30000000,41681.72000000,41752.31000000,508.12196000,2024-01-28 20:59:59.999,21221294.58958080,19176,268.55705000,11217823.02822000,0
1,3393178221,41765.98000000,0.00066000,27.56554680,1706473248350,True,True,2024-01-28 20:00:00,2024-01-28 20:00:00,41785.91000000,41854.30000000,41681.72000000,41752.31000000,508.12196000,2024-01-28 20:59:59.999,21221294.58958080,19176,268.55705000,11217823.02822000,0
2,3393178222,41765.99000000,0.01732000,723.38694680,1706473249825,False,True,2024-01-28 20:00:00,2024-01-28 20:00:00,41785.91000000,41854.30000000,41681.72000000,41752.31000000,508.12196000,2024-01-28 20:59:59.999,21221294.58958080,19176,268.55705000,11217823.02822000,0
3,3393178223,41765.99000000,0.28000000,11694.47720000,1706473249825,False,True,2024-01-28 20:00:00,2024-01-28 20:00:00,41785.91000000,41854.30000000,41681.72000000,41752.31000000,508.12196000,2024-01-28 20:59:59.999,21221294.58958080,19176,268.55705000,11217823.02822000,0
4,3393178224,41765.99000000,0.00018000,7.51787820,1706473249825,False,True,2024-01-28 20:00:00,2024-01-28 20:00:00,41785.91000000,41854.30000000,41681.72000000,41752.31000000,508.12196000,2024-01-28 20:59:59.999,21221294.58958080,19176,268.55705000,11217823.02822000,0


## Cambiar columnas de tipo bool a valores de cero o uno

In [4]:
# Lista de columnas de tipo bool
bool_columns = combined_df.select_dtypes(include=['bool']).columns

# Convertir cada columna bool a int
for column in bool_columns:
    combined_df[column] = combined_df[column].astype(int)

# Verificar los cambios
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   id                            1000 non-null   int64         
 1   price                         1000 non-null   object        
 2   qty                           1000 non-null   object        
 3   quoteQty                      1000 non-null   object        
 4   time                          1000 non-null   int64         
 5   isBuyerMaker                  1000 non-null   int32         
 6   isBestMatch                   1000 non-null   int32         
 7   Time Interval                 1000 non-null   datetime64[ns]
 8   Open Time                     1000 non-null   datetime64[ns]
 9   Open                          1000 non-null   object        
 10  High                          1000 non-null   object        
 11  Low                           1

## Cambiar columnas del tipo Object a Float(float64)

In [5]:
object_columns_to_convert = [
    'price', 'qty', 'quoteQty', 'Open', 'High', 'Low', 
    'Close', 'Volume', 'Quote Asset Volume', 
    'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore'
]

# Convertir cada columna seleccionada a float
for column in object_columns_to_convert:
    combined_df[column] = pd.to_numeric(combined_df[column], errors='coerce')

# Verificar los cambios
combined_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   id                            1000 non-null   int64         
 1   price                         1000 non-null   float64       
 2   qty                           1000 non-null   float64       
 3   quoteQty                      1000 non-null   float64       
 4   time                          1000 non-null   int64         
 5   isBuyerMaker                  1000 non-null   int32         
 6   isBestMatch                   1000 non-null   int32         
 7   Time Interval                 1000 non-null   datetime64[ns]
 8   Open Time                     1000 non-null   datetime64[ns]
 9   Open                          1000 non-null   float64       
 10  High                          1000 non-null   float64       
 11  Low                           1

## Cambiar columnas del tipo fecha a milisegundos

In [6]:
# Lista de columnas de tipo datetime64[ns] que deseas convertir a milisegundos
datetime_columns = ['Time Interval', 'Open Time', 'Close Time']

# Convertir cada columna datetime a milisegundos
for column in datetime_columns:
    combined_df[column] = (combined_df[column].astype('int64') // 10**6)

# Verificar los cambios
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            1000 non-null   int64  
 1   price                         1000 non-null   float64
 2   qty                           1000 non-null   float64
 3   quoteQty                      1000 non-null   float64
 4   time                          1000 non-null   int64  
 5   isBuyerMaker                  1000 non-null   int32  
 6   isBestMatch                   1000 non-null   int32  
 7   Time Interval                 1000 non-null   int64  
 8   Open Time                     1000 non-null   int64  
 9   Open                          1000 non-null   float64
 10  High                          1000 non-null   float64
 11  Low                           1000 non-null   float64
 12  Close                         1000 non-null   float64
 13  Volu

In [7]:
combined_df.sample(7)

,id,price,qty,quoteQty,time,isBuyerMaker,isBestMatch,Time Interval,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Ignore
172,3393178392,41769.99,0.00072,30.074393,1706473264999,1,1,1706472000000,1706472000000,41785.91,41854.3,41681.72,41752.31,508.12196,1706475599999,2.122129e+07,19176,268.55705,1.121782e+07,0
160,3393178380,41769.77,0.00048,20.049490,1706473264086,0,1,1706472000000,1706472000000,41785.91,41854.3,41681.72,41752.31,508.12196,1706475599999,2.122129e+07,19176,268.55705,1.121782e+07,0
929,3393179149,41751.10,0.05046,2106.760506,1706473349615,1,1,1706472000000,1706472000000,41785.91,41854.3,41681.72,41752.31,508.12196,1706475599999,2.122129e+07,19176,268.55705,1.121782e+07,0
737,3393178957,41766.00,0.12186,5089.604760,1706473321070,0,1,1706472000000,1706472000000,41785.91,41854.3,41681.72,41752.31,508.12196,1706475599999,2.122129e+07,19176,268.55705,1.121782e+07,0
567,3393178787,41763.59,0.00021,8.770354,1706473306674,1,1,1706472000000,1706472000000,41785.91,41854.3,41681.72,41752.31,508.12196,1706475599999,2.122129e+07,19176,268.55705,1.121782e+07,0
756,3393178976,41765.80,0.23946,10001.238468,1706473324783,1,1,1706472000000,1706472000000,41785.91,41854.3,41681.72,41752.31,508.12196,1706475599999,2.122129e+07,19176,268.55705,1.121782e+07,0
492,3393178712,41769.06,0.00048,20.049149,1706473302735,1,1,1706472000000,1706472000000,41785.91,41854.3,41681.72,41752.31,508.12196,1706475599999,2.122129e+07,19176,268.55705,1.121782e+07,0


In [8]:
# Cambiar el índice a la columna 'id'
combined_df.set_index('id', inplace=True)

In [9]:
final_DS = combined_df

In [10]:
final_DS.sample(7)

,price,qty,quoteQty,time,isBuyerMaker,isBestMatch,Time Interval,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Ignore
id,,,,,,,,,,,,,,,,,,,
3393178563,41779.91,0.21402,8941.736338,1706473288545,0,1,1706472000000,1706472000000,41785.91,41854.3,41681.72,41752.31,508.12196,1706475599999,2.122129e+07,19176,268.55705,1.121782e+07,0
3393178818,41761.11,0.50000,20880.555000,1706473309859,0,1,1706472000000,1706472000000,41785.91,41854.3,41681.72,41752.31,508.12196,1706475599999,2.122129e+07,19176,268.55705,1.121782e+07,0
3393179216,41752.30,0.00009,3.757707,1706473360954,1,1,1706472000000,1706472000000,41785.91,41854.3,41681.72,41752.31,508.12196,1706475599999,2.122129e+07,19176,268.55705,1.121782e+07,0
3393179215,41752.30,0.00021,8.767983,1706473360954,1,1,1706472000000,1706472000000,41785.91,41854.3,41681.72,41752.31,508.12196,1706475599999,2.122129e+07,19176,268.55705,1.121782e+07,0
3393178598,41779.99,0.00048,20.054395,1706473299179,1,1,1706472000000,1706472000000,41785.91,41854.3,41681.72,41752.31,508.12196,1706475599999,2.122129e+07,19176,268.55705,1.121782e+07,0
3393179212,41751.83,0.00021,8.767884,1706473359878,0,1,1706472000000,1706472000000,41785.91,41854.3,41681.72,41752.31,508.12196,1706475599999,2.122129e+07,19176,268.55705,1.121782e+07,0
3393178314,41765.99,0.00053,22.135975,1706473260548,0,1,1706472000000,1706472000000,41785.91,41854.3,41681.72,41752.31,508.12196,1706475599999,2.122129e+07,19176,268.55705,1.121782e+07,0
